In [1]:
import pandas as pd
import numpy as np
import os
import utils
from datetime import datetime
import importlib

In [2]:
df = pd.concat([pd.read_csv('hka-aqm-am/' + f.removeprefix('._'), skiprows=1, sep=';', engine='python') for f in os.listdir('hka-aqm-am/')])
df.shape


(608036, 18)

In [4]:
importlib.reload(utils)
performance_df = pd.read_csv('model_performances.csv')
d_model=256
nhead=8
num_layers=5 
dropout_pe=0.15
dropout_encoder=0.15
batch_size=1024
learning_rate=0.00031
epochs=25
aggregation_level = 'quarter_hour'
y_feature = 'CO2'
freq = 15 if aggregation_level == 'quarter_hour' else 30 if aggregation_level == 'half_hour' else 60
window_size = 5 * 60 // freq
for y_feature in ['CO2', 'VOC', 'hum', 'tmp']:
    
    model, scaler, rmse, mae, me, mape, train_loss, val_loss, n_features = utils.create_multivariate_transformer_model_for_feature(df, d_model=d_model, nhead=nhead, num_layers=num_layers, dropout_pe=dropout_pe, dropout_encoder=dropout_encoder, batch_size=batch_size, learning_rate=learning_rate, epochs=epochs, y_feature=y_feature, aggregation_level=aggregation_level, window_size=window_size)
    performance_df = performance_df.append({'model_name': 'multivariate_transformer','aggregation_level': aggregation_level, 'y_feature': y_feature, 'n_features': n_features, 'rmse': rmse, 'mae': mae, 'me': me, 'mape': mape, 'd_model': d_model, 'nhead': nhead, 'num_layers': num_layers, 'dropout_pe': dropout_pe,'dropout_encoder': dropout_encoder, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs, 'train_loss': train_loss, 'val_loss': val_loss, 'note': '2 decoder Linear layers with LeakyReLu, shuffled training data'}, ignore_index=True)
    performance_df.to_csv('model_performances.csv', index=False)
        

extended data shape from (395686, 27) to (401930, 27)
training data cutoff:  2023-07-14 03:45:00
date_time_rounded    datetime64[ns]
device_id                    object
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday_sin                 float64
weekday_cos                 float64
month_sin                   float64
month_cos                   float64
time_sin                    float64
time_cos                    float64
semester_SS23               float64
sem

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


          actual   predicted
0     447.000000  420.123641
1     431.000001  421.473891
2     445.000001  430.241168
3     623.999997  661.232737
4     480.000000  448.026895
...          ...         ...
1019  423.999999  444.747124
1020  495.000000  477.513857
1021  510.999999  493.279501
1022  488.000000  472.447271
1023  425.000000  436.473044

[1024 rows x 2 columns]
Epoch 1/25, Training Loss: 0.2660
Epoch 1/25, Validation Loss: 0.1119
          actual   predicted
0     447.000000  418.286876
1     431.000001  419.374151
2     445.000001  435.542233
3     623.999997  675.227402
4     480.000000  473.656121
...          ...         ...
1019  423.999999  437.821871
1020  495.000000  487.357301
1021  510.999999  519.197087
1022  488.000000  473.060118
1023  425.000000  423.066704

[1024 rows x 2 columns]
Epoch 2/25, Training Loss: 0.1587
Epoch 2/25, Validation Loss: 0.0942
          actual   predicted
0     447.000000  411.726863
1     431.000001  421.881990
2     445.000001  439.11324

C:\Users\t-ehm\AppData\Local\Temp\ipykernel_42280\1427812750.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_df = performance_df.append({'model_name': 'multivariate_transformer','aggregation_level': aggregation_level, 'y_feature': y_feature, 'n_features': n_features, 'rmse': rmse, 'mae': mae, 'me': me, 'mape': mape, 'd_model': d_model, 'nhead': nhead, 'num_layers': num_layers, 'dropout_pe': dropout_pe,'dropout_encoder': dropout_encoder, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs, 'train_loss': train_loss, 'val_loss': val_loss, 'note': '2 decoder Linear layers with LeakyReLu, shuffled training data'}, ignore_index=True)


extended data shape from (395686, 27) to (401930, 27)
training data cutoff:  2023-07-14 03:45:00
date_time_rounded    datetime64[ns]
device_id                    object
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday_sin                 float64
weekday_cos                 float64
month_sin                   float64
month_cos                   float64
time_sin                    float64
time_cos                    float64
semester_SS23               float64
sem

...

C:\Users\t-ehm\AppData\Local\Temp\ipykernel_42280\1427812750.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_df = performance_df.append({'model_name': 'multivariate_transformer','aggregation_level': aggregation_level, 'y_feature': y_feature, 'n_features': n_features, 'rmse': rmse, 'mae': mae, 'me': me, 'mape': mape, 'd_model': d_model, 'nhead': nhead, 'num_layers': num_layers, 'dropout_pe': dropout_pe,'dropout_encoder': dropout_encoder, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs, 'train_loss': train_loss, 'val_loss': val_loss, 'note': '2 decoder Linear layers with LeakyReLu, shuffled training data'}, ignore_index=True)


extended data shape from (395686, 27) to (401930, 27)
training data cutoff:  2023-07-14 03:45:00
date_time_rounded    datetime64[ns]
device_id                    object
tmp                         float64
hum                         float64
CO2                         float64
VOC                         float64
vis                         float64
IR                          float64
WIFI                        float64
BLE                         float64
rssi                        float64
channel_rssi                float64
channel_index               float64
spreading_factor            float64
bandwidth                   float64
f_cnt                       float64
isHoliday                   float64
isExamTime                  float64
weekday_sin                 float64
weekday_cos                 float64
month_sin                   float64
month_cos                   float64
time_sin                    float64
time_cos                    float64
semester_SS23               float64
sem

c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
c:\Users\t-ehm\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


         actual  predicted
0     22.230000  22.217415
1     21.940000  21.900738
2     26.840000  26.952173
3     19.840000  18.417838
4     18.840000  16.818404
...         ...        ...
1019  25.630000  25.743582
1020  29.052222  29.540438
1021  25.320000  25.298888
1022  20.570000  19.267527
1023  24.320000  24.459981

[1024 rows x 2 columns]
Epoch 1/25, Training Loss: 0.0816
Epoch 1/25, Validation Loss: 0.0340
         actual  predicted
0     22.230000  22.239312
1     21.940000  21.843627
2     26.840000  26.892044
3     19.840000  19.246664
4     18.840000  17.966701
...         ...        ...
1019  25.630000  25.906853
1020  29.052222  29.274489
1021  25.320000  25.443065
1022  20.570000  19.711486
1023  24.320000  24.310339

[1024 rows x 2 columns]
Epoch 2/25, Training Loss: 0.0251
Epoch 2/25, Validation Loss: 0.0162
         actual  predicted
0     22.230000  22.512785
1     21.940000  22.298735
2     26.840000  26.312048
3     19.840000  20.016384
4     18.840000  19.196246


C:\Users\t-ehm\AppData\Local\Temp\ipykernel_42280\1427812750.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  performance_df = performance_df.append({'model_name': 'multivariate_transformer','aggregation_level': aggregation_level, 'y_feature': y_feature, 'n_features': n_features, 'rmse': rmse, 'mae': mae, 'me': me, 'mape': mape, 'd_model': d_model, 'nhead': nhead, 'num_layers': num_layers, 'dropout_pe': dropout_pe,'dropout_encoder': dropout_encoder, 'batch_size': batch_size, 'learning_rate': learning_rate, 'epochs': epochs, 'train_loss': train_loss, 'val_loss': val_loss, 'note': '2 decoder Linear layers with LeakyReLu, shuffled training data'}, ignore_index=True)


In [ ]:
import torch

device_ids = torch.tensor([0])

device = utils.get_device()

model = utils.load_transformer_model(y_feature="CO2", model_name = 'transformer_multivariate_quarter_hour_26f', device=device)
scaler = utils.load_scaler(y_feature="CO2", model_name = 'transformer_multivariate_quarter_hour_26f')

real_data = utils.load_dataframe(model_name='transformer_multivariate_quarter_hour_26f')
pd.set_option('display.max_columns', None)
real_data.iloc[7:27]

In [ ]:
data_df = real_data.iloc[7:27].drop(columns=['device_id', 'date_time_rounded'])

data_df_scaled = scaler.transform(data_df)

input_data = torch.tensor(data_df_scaled, dtype=torch.float32).view(-1, 20, data_df_scaled.shape[1])
print(input_data.shape)
device_ids = torch.tensor([0])

model.eval()
with torch.no_grad():
    output = model(input_data.to(device), device_ids.to(device))
    print(output)

    prediction = output.cpu().numpy().reshape(-1, 1)
    print(prediction)
    print(prediction.shape)
    zeroes_for_scaler = np.zeros((prediction.shape[0], 25))

    zeroes_for_scaler[:, 2] = prediction  # Insert predicted values into the correct column
    print(zeroes_for_scaler)
    inverse_transformed = scaler.inverse_transform(zeroes_for_scaler)
    predicted_unscaled = inverse_transformed[:, 2].round(2)
    print(predicted_unscaled)
    